In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, from_json
import json
from urllib.parse import unquote
from pyspark.sql.functions import udf
from delta.tables import *
import time
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from datetime import datetime as dt
from pyspark.sql.functions import when,col
from delta.tables import *
from pyspark.sql.functions import *
from delta.tables import *

In [0]:
dbutils.widgets.text("eventhub_name","","");
dbutils.widgets.text("eventhub_namespace","","");
dbutils.widgets.text("access_key_name", "","");
dbutils.widgets.text("access_key","","");
eventhub_name = dbutils.widgets.get("eventhub_name");
eventhub_namespace = dbutils.widgets.get("eventhub_namespace");
access_key_name = dbutils.widgets.get("access_key_name");
access_key = dbutils.widgets.get("access_key");

In [0]:
#Connection to eventhub for streaming data



connection_template = f"Endpoint=sb://{eventhub_namespace}.servicebus.windows.net/;SharedAccessKeyName={access_key_name};SharedAccessKey={access_key};EntityPath={eventhub_name}"
# Start from beginning of stream
startOffset = "-1"

# End at the current time. This datetime formatting creates the correct string format from a python datetime object
endTime = dt.now().strftime("%Y-%m-%dT%H:%M:%S.%fZ")

ehConf = {}
# Create the positions
startingEventPosition = {
  "offset": startOffset,  
  "seqNo": -1,            #not in use
  "enqueuedTime": None,   #not in use
  "isInclusive": True
}

endingEventPosition = {
  "offset": None,           #not in use
  "seqNo": -1,              #not in use
  "enqueuedTime": endTime,
  "isInclusive": True
}


# Put the positions into the Event Hub config dictionary
#ehConf["eventhubs.startingPosition"] = json.dumps(startingEventPosition)
#ehConf["eventhubs.endingPosition"] = json.dumps(endingEventPosition)
ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connection_template)

In [0]:
dict1={}
List1 =["Person.Address","Production.Culture"]
for table in List1:
    Target_table_schema=spark.sql(f"select * from {table}").schema
    dict1.update({table:Target_table_schema})
#Reading Streaming data from eventhub 
read_df = spark.readStream.format("eventhubs").options(**ehConf).load()

#Converting Streaming data from Binary format to String
bodyString=read_df.withColumn("Body", col("body").cast("String")).select("Body")
bodyString.createOrReplaceTempView("jsonStr")
from pyspark.sql.functions import when,col
#Reading Schema from Target Delta Table
#target_table = spark.sql("select concat(Body:source:schema,'.',Body:source:table) from jsonStr").collect
#Target_table_schema=spark.sql(f"select * from {Target_Table_Name}").schema

#Extracting payload Streaming json data in table foramt 
json_payload=spark.sql(f"select Body:before ,Body:after,Body:op,concat(Body:source:schema,'.',Body:source:table) as schemaname,cast(Body:source:ts_ms as double) as basetime from jsonStr")

#Identifying the data is insert, update or delete
changetype = json_payload.withColumn("data", when(json_payload.op == "d",json_payload.before).otherwise(json_payload.after))

schema_type = changetype.withColumn("actual_data_t1",when(changetype.schemaname == List1[0],from_json("data",dict1[List1[0]]))).withColumn("actual_data_t2",when(changetype.schemaname == List1[1],from_json("data",dict1[List1[1]])))
t1_pk="AddressID"
t2_pk="CultureId"

In [0]:
def fullprocess(batchDF, batchId):
    batchDF.persist()
    
    t1_data=batchDF.filter(f"schemaname ='{List1[0]}'").select("op","basetime",col("actual_data_t1.*")).na.drop("all")
    t2_data=batchDF.filter(f"schemaname='{List1[1]}'").select("op","basetime",col("actual_data_t2.*")).na.drop("all")
    t1_delta=DeltaTable.forName(spark, List1[0])
    t2_delta=DeltaTable.forName(spark, List1[1])
    latestChangeForEachKey1 = t1_data.selectExpr(f"{t1_pk} as pkey", "struct(*) as otherCols" ).groupBy("pkey").agg(max("otherCols").alias("latest")).selectExpr("pkey", "latest.*")
    latestChangeForEachKey2 = t2_data.selectExpr(f"{t2_pk} as pkey", "struct(*) as otherCols" ).groupBy("pkey").agg(max("otherCols").alias("latest")).selectExpr("pkey", "latest.*")
    t1_delta.alias("t").merge(latestChangeForEachKey1.alias("s"),f"s.{t1_pk} = t.{t1_pk}").whenMatchedDelete("s.op = 'd'").whenMatchedUpdateAll().whenNotMatchedInsertAll("s.op = 'c' or s.op = 'u'").execute()
    t2_delta.alias("t").merge(latestChangeForEachKey2.alias("s"),f"s.{t2_pk} = t.{t2_pk}").whenMatchedDelete("s.op = 'd'").whenMatchedUpdateAll().whenNotMatchedInsertAll("s.op = 'c' or s.op = 'u'").execute()
    batchDF.unpersist()

In [0]:
bodyString.writeStream \
    .format("delta") \
    .outputMode("update") \
    .option("checkpointLocation", f"/checkpoint/2")\
    .start()

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-1119926258975237> in <module> 
 ----> 1 bodyString . writeStream \ 
 2 . format ( "delta" ) \ 
 3 . outputMode ( "update" ) \ 
 4 . option ( "checkpointLocation" , f"/checkpoint/2" ) \ 
 5 . start ( ) 

 /databricks/spark/python/pyspark/sql/streaming.py in start (self, path, format, outputMode, partitionBy, queryName, **options) 
 1223 self . queryName ( queryName ) 
 1224 if path is None : 
 -> 1225 return self . _sq ( self . _jwrite . start ( ) ) 
 1226 else : 
 1227 return self . _sq ( self . _jwrite . start ( path ) ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 IllegalArgumentException : 'path' is not specified

In [0]:
bodyString.writeStream \
    .format("delta") \
    .foreachBatch(fullprocess) \
    .outputMode("update") \
    .option("checkpointLocation", f"/checkpoint/2")\
    .start()

Out[9]: <pyspark.sql.streaming.StreamingQuery at 0x7ff0cd1dcdc0>

In [0]:
%sql 
select * from Person.address

In [0]:
%sql
select * from Production.Culture